# __텍스트로 텍스트 검색하기__

- 튜토리얼 난이도: ★★☆☆☆
- 읽는데 걸리는 시간: 7분
- 사용 언어: [SQL](https://ko.wikipedia.org/wiki/SQL) (100%)
- 실행 파일 위치: tutorial/thanosql_search/search_text_by_text.ipynb   
- 참고 문서: [Naver sentiment movie corpus v1.0](https://github.com/e9t/nsmc#naver-sentiment-movie-corpus-v10), [단어 임베딩: 어휘의 의미(LEXICAL SEMANTICS)를 인코딩하기](https://tutorials.pytorch.kr/beginner/nlp/word_embeddings_tutorial.html), [컴퓨터가 바라보는 문자](https://heung-bae-lee.github.io/2020/01/16/NLP_01/)

## 튜토리얼 소개

<div class="admonition note">
    <h4 class="admonition-title">텍스트 수치화 기술 이해하기</h4>
    <p>컴퓨터는 사람이 쓰는 언어(자연어)를 곧바로 입력으로 받을 수 없습니다. 때문에 자연어를 기계가 인식할 수 있는 수치 데이터로 변환하는 과정이 필요합니다. 자연어 처리분야에서 임베딩이란 이렇게 사람이 쓰는 자연어를 기계가 이해할 수 있는 숫자형태인 vector로 바꾼 결과 혹은 그 일련의 과정 전체를 의미합니다.</p>
</div>

자연어의 임베딩을 구하는 기법은 크게 통계적인 기법과, 인공신경망 기반 기법으로 구분되고, 세부적으로도 더 나눌 수 있습니다. ThanoSQL에서는 인공신경망 기반의 자가학습모델을 사용하여, 텍스트를 잘 표현하는 벡터를 스스로 학습하는 방법을 제공합니다.

<div class="admonition note">
    <h4 class="admonition-title">본 튜토리얼에서는</h4>
    <p>👉 이번 튜토리얼에서는 <mark style="background-color:#FFD79C">영화 리뷰 데이터</mark>를 사용합니다. 데이터는 영화 리뷰 텍스트와, 라벨값으로 구성되어있지만, 자가학습법을 사용해 훈련하는 예제를 보여드리기 위해 라벨값은 사용하지 않습니다. 10000개의 영화 리뷰 데이터를 학습하여, 입력 문장과 유사한 문장 찾기, 문장의 키워드 추출하기를 해보겠습니다.</p>
</div>

## __0. 데이터 세트 준비__

ThanoSQL의 쿼리 구문을 사용하기 위해서는 [ThanoSQL 워크스페이스](https://docs.thanosql.ai/getting_started/how_to_use_ThanoSQL/#5-thanosql)
에서 언급된 것처럼 API 토큰을 생성하고 아래의 쿼리를 실행해야 합니다.

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<발급받은_API_TOKEN>

### __데이터 세트 준비__

In [2]:
%%thanosql
GET THANOSQL DATASET nsmc_data
OPTIONS (overwrite=True)

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>GET THANOSQL DATASET</strong>" 쿼리 구문을 사용하여 원하는 데이터 세트를 워크스페이스에 저장합니다. </li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>GET THANOSQL DATASET</strong> 에 사용할 옵션을 지정합니다.
        <ul>
            <li>"overwrite": 동일 이름의 데이터 세트가 존재하는 경우 덮어쓰기 가능 유무 설정. True일 경우 기존 데이터 세트는 새로운 데이터 세트로 변경됨 (True|False, DEFAULT: False) </li>
        </ul>
        </li>
    </ul>
</div>

In [3]:
%%thanosql
COPY nsmc_train
OPTIONS (overwrite=True)
FROM "thanosql-dataset/nsmc_data/nsmc_sample_train.csv"

Success


In [4]:
%%thanosql
COPY nsmc_test
OPTIONS (overwrite=True)
FROM "thanosql-dataset/nsmc_data/nsmc_sample_test.csv"

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>COPY</strong>" 쿼리 구문을 사용하여 DB에 저장 할 데이터 세트명을 지정합니다. </li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>COPY</strong> 에 사용할 옵션을 지정합니다.
        <ul>
            <li>"overwrite": 동일 이름의 데이터 세트가 DB상에 존재하는 경우 덮어쓰기 가능 유무 설정. True일 경우 기존 데이터 세트는 새로운 데이터 세트로 변경됨 (True|False, DEFAULT: False) </li>
        </ul>
        </li>
    </ul>
</div>

## __1. 데이터 세트 확인__

텍스트 수치화 모델을 만들기 위해 ThanoSQL 워크스페이스 DB에 저장되어 있는 <mark style="background-color:#FFEC92">nsmc_train</mark> 테이블을 사용합니다. <mark style="background-color:#FFEC92">nsmc_train</mark> 테이블은 <mark style="background-color:#FFD79C">NAVER Sentiment Movie Corpus</mark> 영화 리뷰 데이터 및 라벨 정보의 일부가 담겨 있는 테이블입니다. 아래의 쿼리문을 실행하고 테이블의 내용을 확인합니다.

In [5]:
%%thanosql
SELECT *
FROM nsmc_train
LIMIT 5

,id,document,label
0,9465891,저퀄리티. 뭐하나 장점이 없음.,0
1,9792014,10점 만점에 1점 !!! 최악이네 하아...,0
2,3795348,누가 억지눈물 잘흘리나 자기네들끼리 대결하는 작품,0
3,1129393,신도 싫고 인간도 싫다,1
4,4678610,추억의 명작만화,1


<div class="admonition note">
    <h4 class="admonition-title">데이터 테이블 이해하기</h4>
    <p><mark style="background-color:#FFEC92">nsmc_train</mark> 테이블은 아래와 같은 정보를 담고 있습니다.</p>
    <ul>
        <li><mark style="background-color:#D7D0FF">id</mark>: 데이터의 식별 번호</li>
        <li><mark style="background-color:#D7D0FF">document</mark>: 영화 리뷰 데이터</li>
        <li><mark style="background-color:#D7D0FF">label</mark>: 라벨값</li>
    </ul>
</div>

## __2. 텍스트 수치화 모델 생성__

이전 단계에서 확인한 <mark style="background-color:#FFEC92">nsmc_train</mark> 테이블을 사용하여 텍스트 수치화 모델을 만듭니다. 아래의 쿼리 구문을 실행하여 <mark style="background-color:#E9D7FD">nsmc_text_search_model</mark>이라는 이름의 모델을 만듭니다.  
(쿼리 실행 시 예상 소요 시간: 2 min)

In [6]:
%%thanosql
BUILD MODEL nsmc_text_search_model
USING SBERTKo
OPTIONS (
    text_col="document",
    overwrite=True
)
AS
SELECT *
FROM nsmc_train

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>BUILD MODEL</strong>" 쿼리 구문을 사용하여 <mark style="background-color:#E9D7FD">nsmc_text_search_model</mark> 이라는 모델을 만들고 학습시킵니다.</li>
        <li>"<strong>USING</strong>" 쿼리 구문을 통해 베이스 모델로 <mark style="background-color:#E9D7FD">SBERTKo</mark> 모델을 사용할 것을 명시합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 모델 생성에 사용할 옵션을 지정합니다.
        <ul>
            <li>"text_col": 데이터 테이블에서 영화 리뷰 데이터를 담은 컬럼 (Default: "text")</li>
            <li>"epochs": 텍스트 수치화 모델을 생성하기 위한 데이터 세트 학습 횟수 (int, DEFAULT: 1)</li>
            <li>"batch_size": 한 번의 예측에서 읽는 데이터 세트 묶음의 크기 (int, DEFAULT: 16)</li> 
            <li>"learning_rate": 모델의 학습률 (float, DEFAULT: 3e-5)</li> 
            <li>"train": False일 경우 사전훈련된 모델을 추가로 학습하지 않고 그대로 사용 (True|False, DEFAULT: True)</li> 
            <li>"overwrite": 동일 이름의 모델이 존재하는 경우 덮어쓰기 가능 유무 설정. True일 경우 기존 모델은 새로운 모델로 변경됨 (True|False, DEFAULT: False)</li>
        </ul>
        </li>
    </ul>
</div>

다음 "__CONVERT USING__ " 쿼리 구문을 실행하여 `nsmc_test` 텍스트 데이터들을 수치화 합니다. 수치화 된 결과는 `nsmc_test` 테이블에 사용자가 옵션으로 지정한 이름(Default: <mark style="background-color:#D7D0FF ">convert_result</mark>)의 컬럼에 저장됩니다.

In [7]:
%%thanosql
CONVERT USING nsmc_text_search_model
OPTIONS (
    text_col="document",
    table_name="nsmc_test",
    batch_size=32,
    column_name="convert_result"
    )
AS 
SELECT *
FROM nsmc_test

,id,document,label,convert_result
0,9181084,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,"[0.0037510623, -0.0031921342, -0.01413603, -0...."
1,6318649,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,"[-0.061903417, -0.016521271, 0.007930033, -0.0..."
2,5718332,유쾌하고 신나는 스피드 코미디.,1,"[-0.006741981, -0.05358216, 0.04350551, -0.006..."
3,8063675,정말 따뜻했던 드라마ㅠㅠ,1,"[0.030807665, -0.00572762, 0.012884197, -0.027..."
4,10229366,솔직히 배우들의 연기는 나름 괜찮았다 하지만 냉정하게 영화자체만을 놓고보면 별다른 ...,0,"[-0.002742508, -0.020427909, 0.03971886, -0.01..."
...,...,...,...,...
1995,5382227,30년간 본 수천편의 드라마중 최고는 이 작품,1,"[-0.018247725, -0.01442731, 0.015619117, -0.03..."
1996,4584773,빛돌을 이길 5명의 용사들의 패배,1,"[-0.042145733, -0.0331033, 0.012950524, 0.0350..."
1997,2559484,빌머레이의 굳은 표정에 모든것이 살아있다.. 짐자무시 최고!,1,"[-0.030894795, 0.0091382265, -0.032140985, 0.0..."
1998,9867081,이런영화에 투자하지 않는것이 진정한 한국영화의 발전을위한 초석,0,"[0.008489536, 0.037255745, 0.042381987, 0.0331..."


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>CONVERT USING</strong>" 쿼리 구문은 <code>nsmc_text_search_model</code>을 텍스트 수치화를 위한 알고리즘으로 사용합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 텍스트 수치화 시 필요한 변수들을 정의합니다.
        <ul>
            <li>"text_col": 데이터 테이블에서 영화 리뷰 데이터를 담은 컬럼 (Default: "text")</li>
            <li>"table_name": ThanoSQL 워크스페이스 DB 내에 저장될 테이블 이름을 정의합니다.</li>
            <li>"batch_size": 한 번의 예측에서 읽는 데이터 세트 묶음의 크기 (int, DEFAULT: 16)</li>
            <li>"column_name": 데이터 테이블에서 수치화된 결과를 담을 컬럼 이름을 정의합니다. (Default: "convert_result")</li>
        </ul>
        </li>
    </ul>
</div>

## __3. 텍스트 수치화 모델을 사용해서 유사한 문서 검색하기__

이번 단계에서는 <mark style="background-color:#E9D7FD">nsmc_text_search_model</mark> 텍스트 수치화 모델과 테스트 테이블을 사용하여 유사한 문서를 검색합니다.

In [8]:
%%thanosql
SELECT document, label, score 
FROM (
    SEARCH TEXT text="영화 내용이 불만족스러웠다"
    USING nsmc_text_search_model
    OPTIONS (
        emb_col="convert_result",
        column_name="score"
        )
    AS 
    SELECT * 
    FROM nsmc_test
    )
ORDER BY score DESC 
LIMIT 10

,document,label,score
0,잔잔한 영화 보기 불편할수도 있음,1,0.659124
1,영화로서는 많이 아쉬운 듯..,0,0.642425
2,너무 작위적인 스토리 전개..실망스러운 영화ㅠ,0,0.642100
3,시나리오는 한없이 후진데다가 썩은 동선이 가득한 영화,0,0.633289
4,가벼운 마음으로 웃으면서 영화를 따라가다보니 어느새 끝나있는 영화,1,0.625073
5,정말 싱겁다 영화를 다봣어도 보다만 느낌이다,0,0.623635
6,"부족한 감이 있었고, 암울했다.",0,0.618519
7,내가 생각했던 스토리와 조금 달랐지만 신선하고 독특했던 영화였다.,1,0.616638
8,"평점보고 기대해서 그런지 실망스런 영화 ,쏘쏘한 영화",0,0.610020
9,무관심했던 나를 반성케 한 영화,1,0.603797


In [9]:
%%thanosql
SELECT document, label, score
FROM (
    SEARCH TEXT text="기분이 좋아지는 작품"
    USING nsmc_text_search_model
    OPTIONS (
        emb_col="convert_result",
        column_name="score"
        )
    AS
    SELECT *
    FROM nsmc_test
    )
ORDER BY score DESC
LIMIT 10

,document,label,score
0,보고나면 기분 좋은 영화,1,0.773304
1,감동적인 영화,1,0.719860
2,너무 멋있는 영화. 환한 느낌을 준다,1,0.708152
3,몰입도 잘되고 좋은 영화,1,0.693896
4,오래됐지만 재미있는 영화,1,0.688140
5,뻔한 스토리지만 과거 추억을 생각나게 해주는 영화 재미있었어요,1,0.678594
6,멋진영화,1,0.678019
7,인상 깊은 영화다~,1,0.671168
8,오랜만에 좋은 영화를 보았다.,1,0.667201
9,추억속의 영화 ㅎ 정말 재밌게 봤습니다!,1,0.663330


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>SEARCH TEXT [image|audio|video|text|keyword]</strong>" 쿼리 구문은 검색하고자 하는 이미지|오디오|비디오|텍스트|키워드 데이터를 정의합니다.</li>
        <li>"<strong>USING</strong>"은 텍스트 수치화에 사용할 모델을 정의합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 텍스트 검색 시 필요한 변수들을 정의합니다.
        <ul>
                <li>"emb_col": 데이터 테이블에서 수치화된 결과를 담은 컬럼</li>
                <li>"column_name": 데이터 테이블에서 검색 결과를 담을 컬럼 이름을 정의합니다. (default: "search_result")</li>
        </ul>
        </li>
        <li>"<strong>AS</strong>" 쿼리 구문은 검색에 사용할 임베딩 테이블을 정의합니다. <code>nsmc_test</code> 테이블을 사용합니다.</li>
    </ul>
</div>

## __4. 텍스트 수치화 모델을 사용해서 문서에서 키워드 추출하기__

이번 단계에서는 <mark style="background-color:#E9D7FD">nsmc_text_search_model</mark> 텍스트 수치화 모델과 테스트 테이블을 사용하여 유사한 문서에서 키워드 추출합니다. 

In [10]:
%%thanosql
SEARCH KEYWORD
USING nsmc_text_search_model
OPTIONS (
    text_col="document",
    ngram_range=[1, 3],
    use_stopwords=True
    )
AS 
SELECT * 
FROM nsmc_test
LIMIT 10 OFFSET 40

,id,document,label,convert_result,keyword
0,7928591,"일단 엠씨 역량 부족이 가장 큰 문제인듯. 시간 없다고, 재미없다고 게스트 말 끊는...",0,"[0.012267594, -0.01771399, -0.010869383, -0.00...","{'keyword': ['라고 끝 엠씨가노답임', '방 왜 ᆷ', '가장 크 문제'..."
1,8380896,방금 슬쩍 봤는데 '인삼' 처럼 생긴 '산삼' 암수가 서로 토닥이다가 주인공에게 잡...,0,"[-0.042678047, -0.02154563, -0.0577272, -0.022...","{'keyword': ['산삼 암수 서로', '주인공 잡히 ᆷ', '는데 인삼 처럼..."
2,8914833,신세계에 비교하는 거 진심 빡친다. 신성모독,1,"[0.026429567, -0.014425963, -0.0066649574, 0.0...","{'keyword': ['빡 치 신성모독', '거 진심', '진심 빡 치', '신세..."
3,9702804,킬링타임 최고다 열자채워,1,"[0.019156558, -0.025161155, -0.042373788, 0.02...","{'keyword': ['킬링타임 최고 다', '킬링타임', '최고 다 열', '열..."
4,10097178,명불허전 선동영화 10자,0,"[-0.010032367, -0.020877095, 0.02225531, -0.04...","{'keyword': ['명불허전 선동 영화', '명불허전 선동', '명불허전', ..."
5,9245590,결말이 마음에 든다. 소설을 먼저 읽었던 터라 결말이 마음에 든다. 세상의 모든 정...,1,"[-0.009547141, -0.03575876, 0.022106346, -0.00...","{'keyword': ['결말 마음 들', '모든 정혜 힘내', '들 세상 모든',..."
6,4227156,기대했는데.. 시나리오는 ...어떻게 결말이 그렇지?..요즘 막장 드라마 영향인가..,0,"[0.03098442, -0.05774053, 0.033200644, 0.01355...","{'keyword': ['시나리오 어떻 결말', '드라마 영향 ᆫ가', '요즘 막장..."
7,7313784,인기연예인 몇명 출현한다고 재미있는 영화냐 잡것들아긴급조치19호가 그렇게 만들고 망...,0,"[0.025257412, -0.030326752, 0.02663638, 0.0056...","{'keyword': ['영화 냐 잡것', '인기 연예인 몇', '출현 ᆫ다고', ..."
8,3582981,좋았음!!,1,"[0.0419066, -0.00032061862, -0.037271593, 0.03...","{'keyword': ['좋', '좋 음', '음'], 'score': [0.547..."
9,9906269,멜로와 스릴러를 이렇게 조합할수 있다니 ...,1,"[-0.022876328, -0.071984485, 0.009473692, 0.00...","{'keyword': ['멜로 스릴러', '스릴러 이렇 조합', '조합', '이렇 ..."


In [11]:
%%thanosql
SELECT document, label, keyword -> 'keyword' AS keywords, keyword -> 'score' AS score
FROM (
    SEARCH KEYWORD 
    USING nsmc_text_search_model
    OPTIONS (
        text_col="document",
        use_stopwords=True
        )
    AS 
    SELECT * 
    FROM nsmc_test
    LIMIT 10
)

,document,label,keywords,score
0,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,"[영화 네요, 꼭 번, 네요, 번 영화, 꼭]","[0.6508, 0.5355, 0.5246, 0.4709, 0.4693]"
1,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,"[권선징악 안드로메다, 빙의 니, ~여주 남자, 질 나쁘, 라인 작가]","[0.4953, 0.3959, 0.3657, 0.2388, 0.2272]"
2,유쾌하고 신나는 스피드 코미디.,1,"[스피드 코미디, 유쾌, 코미디, 스피드, 신 나]","[0.7569, 0.6539, 0.5764, 0.574, 0.4409]"
3,정말 따뜻했던 드라마ㅠㅠ,1,"[정말 따뜻하, 따뜻하 드라마, 정말, 따뜻하, 드라마]","[0.8149, 0.804, 0.7173, 0.7024, 0.5407]"
4,솔직히 배우들의 연기는 나름 괜찮았다 하지만 냉정하게 영화자체만을 놓고보면 별다른 ...,0,"[연출력 그냥, 전개, 별다르 특색, 살인마 주인공, 하지만 냉정]","[0.4769, 0.3703, 0.3028, 0.3021, 0.2695]"
5,느와르에 멜로에 신파극까지······. 짬뽕이 아니라 꿀꿀이죽,0,"[신파극 짬뽕, 짬뽕 꿀꿀이죽, 느와르, 짬뽕, 멜로]","[0.5395, 0.4998, 0.4831, 0.3861, 0.3857]"
6,다시 보고 싶네요~,1,"[다시 싶, 싶 네요, 다시, 네요, 싶]","[0.6543, 0.6535, 0.6249, 0.5935, 0.3829]"
7,짱이야... 진짜 극장을 나오는데 멋진 소설을 한권 읽은 듯한 기분을 느낌. 재미는...,1,"[감동 철철, 짱 야, 영화 꼭, 재미 물론, 권 읽]","[0.5069, 0.4895, 0.4673, 0.3759, 0.3449]"
8,유일하게 아들과 같이볼수있는 만화영화,1,"[만화 영화, 아들 같이, 유일 아들, 유일, 같이 있]","[0.534, 0.4989, 0.4354, 0.3868, 0.3645]"
9,남자라면 봐라 꼭봐라 두번봐라 세번봐라 계속봐라,1,"[계속 어라, 세 번, 꼭, 남자 라면, 보]","[0.4975, 0.3957, 0.368, 0.2101, 0.1979]"


In [12]:
%%thanosql 
SELECT * FROM (
    SELECT document, label, json_array_elements(keyword -> 'keyword') AS keywords, (json_array_elements(keyword -> 'score'))::text::float AS score
        FROM (
            SEARCH KEYWORD 
            USING nsmc_text_search_model
            OPTIONS (
                text_col="document",
                use_stopwords=True
                )
            AS 
            SELECT * 
            FROM nsmc_test
            LIMIT 10
        )
    ) 
WHERE score > 0.5

,document,label,keywords,score
0,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,영화 네요,0.6508
1,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,꼭 번,0.5355
2,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,네요,0.5246
3,유쾌하고 신나는 스피드 코미디.,1,스피드 코미디,0.7569
4,유쾌하고 신나는 스피드 코미디.,1,유쾌,0.6539
5,유쾌하고 신나는 스피드 코미디.,1,코미디,0.5764
6,유쾌하고 신나는 스피드 코미디.,1,스피드,0.5740
7,정말 따뜻했던 드라마ㅠㅠ,1,정말 따뜻하,0.8149
8,정말 따뜻했던 드라마ㅠㅠ,1,따뜻하 드라마,0.8040
9,정말 따뜻했던 드라마ㅠㅠ,1,정말,0.7173


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>SEARCH KEYWORD</strong>" 쿼리 구문은 키워드를 검색하기 위한 알고리즘으로 사용합니다.</li>
        <li>"<strong>USING</strong>"은 텍스트 수치화에 사용할 모델을 정의합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 텍스트 수치화 시 필요한 변수들을 정의합니다.
            <ul>
                <li>"lang": en, ko</li>
                <li>"text_col": 텍스트가 있는 열 이름</li>
                <li>"ngram_range": 키워드의 최소 단어 수와 최대 단어수. 예) [1, 3]. 대부분의 상황에서 키워드는 최대 단어 수에 맞춰 추출됩니다 (list[int, int], DEFAULT: [1, 2])</li>
                <li>"top_n": 추출할 키워드의 수, 유사도가 높은 순서대로 (int, DEFAULT: 5)</li>
                <li>"diversity": 추출될 키워드의 다양성. 높을 수록 기존에 추출된 키워드와 유사한 키워드는 다시 추출되지 않습니다. 0 <= diversity <= 1 (float, DEFAULT: 0.5)</li>
                <li>"use_stopwords": 큰 의미가 없는 단어(불용어)를 제외할 지 여부 (True|False, DEFAULT: True)</li>
                <li>"threshold": 추출할 키워드의 유사도 수치의 최소값 (float, DEFAULT: 0.0)</li>
            </ul>
            </li>
        <li>"<strong>AS</strong>" 쿼리 구문은 검색에 사용할 임베딩 테이블을 정의합니다. <code>nsmc_test</code> 테이블을 사용합니다.</li>
    </ul>
</div>

## __5. 두 방법 결합하여 사용하기__

In [13]:
%%thanosql
SEARCH KEYWORD
USING nsmc_text_search_model
OPTIONS (
    text_col="document",
    ngram_range=[1, 3],
    use_stopwords=True
    )
AS (
    SELECT document, label, score
    FROM (
        SEARCH TEXT text="가볍게 볼 수 있는 코미디 영화"
        USING nsmc_text_search_model
        OPTIONS (
            emb_col = "convert_result",
            column_name="score"
            )
        AS 
        SELECT * 
        FROM nsmc_test
        )
    ORDER BY score DESC 
    LIMIT 10
)

,document,label,score,keyword
0,순수하게 보면 참으로 재밌는 영화,1,0.745830,"{'keyword': ['참으로 재밌 영화', '순수 보 참으로', '보 참으로',..."
1,오래됐지만 재미있는 영화,1,0.692706,"{'keyword': ['오래되 재미있 영화', '재미있 영화', '오래되 재미있'..."
2,보고나면 기분 좋은 영화,1,0.692283,"{'keyword': ['기분 좋 영화', '좋 영화', '기분 좋', '영화', ..."
3,감동적인 영화,1,0.684076,"{'keyword': ['감동 영화', '감동', '영화'], 'score': [0..."
4,코미디로도 사람의 마음을 울릴수 있느 영화,1,0.678831,"{'keyword': ['울리 있느 영화', '코미디 마음 울리', '코미디 마음'..."
5,시간가는줄 모르고 잼있게봤다 볼만한 영화,1,0.670244,"{'keyword': ['보 만 영화', '모르 잼 보', '시간 가', '줄', ..."
6,레전드란 말이 어울리는 몇 안되는 영화중 하나,1,0.661960,"{'keyword': ['레전드 란 어울리', '몇 안 영화', '어울리 몇 안',..."
7,가벼운 마음으로 웃으면서 영화를 따라가다보니 어느새 끝나있는 영화,1,0.648359,"{'keyword': ['어느새 끝나 영화', '마음 웃 으면서', '영화 따라가 ..."
8,간만에 본 한국영화 중 수작중의 수작,1,0.647729,"{'keyword': ['간만에 한국 영화', '한국 영화 수작', '영화 수작 수..."
9,모든것이 흥미로운 영화였다.,1,0.640093,"{'keyword': ['모든 흥미', '모든 흥미 롭', '흥미 롭 영화', '모..."


In [14]:
%%thanosql
SELECT document, label, keyword -> 'keyword' AS keywords, keyword -> 'score' AS score
FROM ( 
    SEARCH KEYWORD
    USING nsmc_text_search_model
    OPTIONS (
        text_col="document",
        ngram_range=[1, 3],
        use_stopwords=True
        )
    AS (
        SELECT document, label, score
        FROM (
            SEARCH TEXT text="가볍게 볼 수 있는 코미디 영화"
            USING nsmc_text_search_model
            OPTIONS (
                emb_col = "convert_result",
                column_name="score"
                )
            AS 
            SELECT * 
            FROM nsmc_test
            )
        ORDER BY score DESC 
        LIMIT 10
    )
)

,document,label,keywords,score
0,순수하게 보면 참으로 재밌는 영화,1,"[참으로 재밌 영화, 순수 보 참으로, 보 참으로, 영화, 순수]","[0.7928, 0.6529, 0.5572, 0.5498, 0.5255]"
1,오래됐지만 재미있는 영화,1,"[오래되 재미있 영화, 재미있 영화, 오래되 재미있, 영화, 오래되]","[0.7788, 0.7097, 0.6668, 0.6538, 0.653]"
2,보고나면 기분 좋은 영화,1,"[기분 좋 영화, 좋 영화, 기분 좋, 영화, 나 기분]","[0.8424, 0.7348, 0.715, 0.6244, 0.579]"
3,감동적인 영화,1,"[감동 영화, 감동, 영화]","[0.885, 0.7347, 0.6884]"
4,코미디로도 사람의 마음을 울릴수 있느 영화,1,"[울리 있느 영화, 코미디 마음 울리, 코미디 마음, 마음, 영화]","[0.662, 0.6117, 0.4975, 0.4285, 0.4237]"
5,시간가는줄 모르고 잼있게봤다 볼만한 영화,1,"[보 만 영화, 모르 잼 보, 시간 가, 줄, 가 줄 모르]","[0.5494, 0.476, 0.4309, 0.3598, 0.3372]"
6,레전드란 말이 어울리는 몇 안되는 영화중 하나,1,"[레전드 란 어울리, 몇 안 영화, 어울리 몇 안, 영화 하나, 하나]","[0.5986, 0.5929, 0.4756, 0.4671, 0.3399]"
7,가벼운 마음으로 웃으면서 영화를 따라가다보니 어느새 끝나있는 영화,1,"[어느새 끝나 영화, 마음 웃 으면서, 영화 따라가 보, 보 니 어느새, 가볍]","[0.71, 0.4958, 0.4768, 0.424, 0.3438]"
8,간만에 본 한국영화 중 수작중의 수작,1,"[간만에 한국 영화, 한국 영화 수작, 영화 수작 수작, 수작 수작, 영화]","[0.718, 0.6774, 0.6634, 0.5333, 0.494]"
9,모든것이 흥미로운 영화였다.,1,"[모든 흥미, 모든 흥미 롭, 흥미 롭 영화, 모든, 롭 영화]","[0.6237, 0.6141, 0.5517, 0.5436, 0.5123]"


In [15]:
%%thanosql
SELECT * 
FROM 
    (SELECT document, label, json_array_elements(keyword -> 'keyword') AS keywords, (json_array_elements(keyword -> 'score'))::text::float AS score
    FROM (
        SEARCH KEYWORD
        USING nsmc_text_search_model
        OPTIONS (
            text_col="document",
            ngram_range=[1, 3],
            use_stopwords=True
            )
        AS (
            SELECT document, label, score
            FROM (
                SEARCH TEXT text="최고의 액션 영화"
                USING nsmc_text_search_model
                OPTIONS (
                    emb_col = "convert_result",
                    column_name="score"
                    )
                AS 
                SELECT * 
                FROM nsmc_test
                WHERE document LIKE '%%판타지%%'
                )
            ORDER BY score DESC
            LIMIT 10
        )
    )
)
WHERE score > 0.3

,document,label,keywords,score
0,정말 재밌다.이런게 바로 판타지모험영화의 현실이다.,1,바로 판타지 모험,0.6523
1,정말 재밌다.이런게 바로 판타지모험영화의 현실이다.,1,이런 바로 판타지,0.6060
2,정말 재밌다.이런게 바로 판타지모험영화의 현실이다.,1,정말 재밌 이런,0.5733
3,정말 재밌다.이런게 바로 판타지모험영화의 현실이다.,1,영화,0.4376
4,정말 재밌다.이런게 바로 판타지모험영화의 현실이다.,1,현실,0.4012
5,30년 전의 퓨전 판타지라니..그냥 찬양해야됨,1,그냥 찬양 ᆷ,0.6318
6,30년 전의 퓨전 판타지라니..그냥 찬양해야됨,1,퓨전 판타지 라니,0.5863
7,30년 전의 퓨전 판타지라니..그냥 찬양해야됨,1,라니 그냥,0.4737
8,30년 전의 퓨전 판타지라니..그냥 찬양해야됨,1,찬양,0.4026
9,1700만 1700만 해되서 봤다가 욕만하며 참고 참으며 보다 결국 사분의 일은 스...,0,참 으며 보다,0.3967


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>SEARCH TEXT [image|audio|video|text|keyword]</strong>" 쿼리 구문은 검색하고자 하는 이미지|오디오|비디오|텍스트|키워드 데이터를 정의합니다.</li>
        <li>"<strong>SEARCH KEYWORD</strong>" 쿼리 구문은 키워드를 검색하기 위한 알고리즘으로 사용합니다.</li>
        <li>"<strong>USING</strong>"은 텍스트 수치화에 사용할 모델을 정의합니다.</li>
        <li>"<strong>AS</strong>" 쿼리 구문은 검색에 사용할 임베딩 테이블을 정의합니다. <code>nsmc_test</code> 테이블을 사용합니다.</li>
    </ul>
</div>

## __6. 튜토리얼을 마치며__

이번 튜토리얼에서는 `nsmc` 영화 리뷰 데이터를 사용하여 텍스트 수치화와 수치화 결과를 바탕으로 한 유사 텍스트 검색, 키워드 추출을 진행해 보았습니다. 이번 튜토리얼에서는 모델의 수치화 성능보다는 작동 위주의 설명으로 진행하였습니다. 나만의 텍스트 수치화모델을 만들어 다양한 형태의 비정형 데이터 세트에 검색 기능을 추가하고 Auto-ML 기법을 이용한 나만의 모델을 배포할 수 있습니다.
<br>
다음 단계에서 텍스트 수치화 모델의 다양한 "__OPTIONS__" 쿼리 구문과 학습 방법을 더욱 심도있게 다뤄봅니다. 나만의 정확한 텍스트 수치화 모델 구축방법에 대해 더욱 자세히 알고 싶다면 다음 튜토리얼들을 진행 해보세요.

* [나만의 데이터 업로드하기](https://docs.thanosql.ai/getting_started/data_upload/)
* [중급 유사 텍스트 검색 모델 만들기]

<div class="admonition tip">
    <h4 class="admonition-title">나만의 서비스를 위한 모델 배포 관련 문의</h4>
    <p>ThanoSQL을 활용해 나만의 모델을 만들거나, 나의 서비스에 적용하는데 어려움이 있다면 언제든 아래로 문의주세요😊</p>
    <p>유사 텍스트 검색 모델 구축 관련 문의: contact@smartmind.team</p>
</div>